In [27]:
import pandas as pd
import csv
import os
from dotenv import load_dotenv
import aiohttp
import asyncio
from aiohttp import ClientSession
from pprint import pprint
from dataclasses import dataclass
import json


In [50]:
# Set the competition to get the players from (e.g. 'L1', 'GB1', 'ES1', 'IT1', 'FR1')
set_competition = 'IT1'

In [51]:
load_dotenv()
api_key = os.getenv('API_KEY')
api_host = os.getenv('API_HOST')
competetions = ['L1', 'GB1', 'ES1', 'IT1', 'FR1']
seasons = list(range(2021, 2024))
next_delay = 0.3
players = {}

In [52]:
teams_per_season = {}

for league in competetions:
    teams_per_season[league] = {}
    for season in seasons:
        teams_per_season[league][season] = {}

In [53]:
with open('teams_per_season2021-2023.csv') as f:
    for i in csv.reader(f, delimiter=';'):
        str = i[2]
        l = []
        for j in str[1:-1].split(','):
            l.append(int(j))
        teams_per_season[i[0]][int(i[1])] = l
    

In [54]:
error = []

In [55]:
async def get_players(season, team, session):
    global next_delay
    global players
    
    next_delay += 0.3
    await asyncio.sleep(next_delay)
    
    url = 'https://transfermarket.p.rapidapi.com/clubs/get-squad'
    querystring = {'id':team, 'saison_id':season}
    headers = {
        'X-RapidAPI-Key': api_key,
        'X-RapidAPI-Host': api_host
        }
    
    async with session.get(url, headers=headers, params=querystring) as response:
        if response.status == 200:
            dict = await response.json()
            if dict.get('squad') is not None:
                for p in dict['squad']:
                    key = int(p.get('id'))
                    players[key] = {}
        if response.status != 200:
            error.append([season, team, response])
            print(f'Error: {season}, {team}, {response.status}')
    
async def get_players_of_teams(seasons, teams_per_season, set_competition):
    
    async with ClientSession() as session:
        tasks = []
        for season in teams_per_season[set_competition]:
            for team in teams_per_season[set_competition][season]:
                task = asyncio.ensure_future(get_players(season, team, session))
                tasks.append(task)
        print(f'{len(tasks)} Tasks created')
        await asyncio.gather(*tasks)
    
    global next_delay
    next_delay = 0.3

In [56]:
await get_players_of_teams(seasons, teams_per_season, set_competition)

60 Tasks created


In [57]:
rest = {
    set_competition:{}
}

In [58]:
error[0]

IndexError: list index out of range

In [ ]:
for i in error:
    if i[0] not in rest[set_competition]:
        rest[set_competition][i[0]] = []
    rest[set_competition][i[0]].append(i[1])

In [ ]:
await get_players_of_teams(seasons, rest, set_competition)

0 Tasks created


In [59]:
len(players)

1177

In [ ]:
pd.DataFrame(players).T.to_csv(f'players_{set_competition}-2021-2023.csv', header=False)